In [5]:
from database import SessionLocal
from sql_alchemy_orm import User, Role  


session = SessionLocal()
# Query all users
users = session.query(User).all()
for user in users:
    print(f"{user.username} has a role of {user.role.name}")

alice123 has a role of Admin
bob456 has a role of Admin
charlie789 has a role of Editor
david000 has a role of Viewer
eve999 has a role of Viewer


In [6]:
# Query user by username
user = session.query(User).filter(User.username == 'john_doe').first()

if user:
    print(f"User found: {user.name}, {user.email}")
else:
    print("User not found.")


User not found.


### Filtering with `filter` (WHERE clause):

In [7]:
user = session.query(User).filter(User.username == 'alice123').first()
user.name

'Alice'

### Using `filter_by` (for simpler filters):
`filter_by()` is another method to apply filters, which is a bit simpler compared to `filter()`. It's mainly used for equality checks.

In [8]:
user = session.query(User).filter_by(username='alice123').first()

if user:
    print(f"User found: {user.name}, {user.email}")
else:
    print("User not found.")

User found: Alice, alice@email.com


Combining Filters with `and_` or `or_`

In [9]:
# Example: Query users by both username and role ID
from sqlalchemy import and_, or_, func

user = session.query(User).filter(and_(User.username == 'john_doe', User.role_id == 1)).first()

if user:
    print(f"User found: {user.name}, {user.email}")
else:
    print("User not found.")

User not found.


In [10]:
user = session.query(User).filter(and_(User.username == 'alice123', User.role_id == 1)).first()
if user:
    print(f"User found: {user.name}, {user.role_id}")
else:
    print("User not found.")

User found: Alice, 1


### `.first()` vs `.all()`
- `.first()` → Returns only the first matching record (if any) or None if no records match.
- `.all()` → Returns a list of all matching records.

In [12]:
# Query user by username or email
user = session.query(User).filter(or_(User.username == 'alice123', User.email == 'bob@email.com')).first()

if user:
    print(f"User found: {user.name}, {user.username}")
else:
    print("User not found.")

User found: Alice, alice123


In [14]:
# Query user by username or email
users = session.query(User).filter(or_(User.username == 'alice123', User.email == 'bob@email.com')).all()
if users:
    for user in users:
        print(f"User found: {user.name}, {user.username}")
else:
    print("No users found.")

User found: Alice, alice123
User found: Bob, bob456


In [15]:
## Using the startswith() and endswith() methods
users = session.query(User).filter(User.username.startswith('al')).all()

for user in users:
    print(f"User found: {user.name}, {user.username}")

User found: Alice, alice123


In [16]:
## find all users with 'gmail' in their email address
users = session.query(User).filter(User.email.endswith('@gmail.com')).all()

for user in users:
    print(f"User found: {user.name}, {user.email}")


`like()` – Pattern matching using `%` (wildcards)

In [17]:
users = session.query(User).filter(User.username.like('%bob%')).all()

for user in users:
    print(f"User found: {user.name}, {user.username}")

User found: Bob, bob456


- `%` matches any number of characters.
- This finds usernames containing `"bob"` anywhere (e.g., `"big_bob"`, `"robert_bobson"`).


In [19]:
users = session.query(User).filter(
    and_(
        User.username.startswith('a'),
        User.email.endswith('@yahoo.com')
    )
).all()
if users:
    for user in users:
        print(f"User found: {user.name}, {user.username}, {user.email}")
else:
    print("No users found.")

No users found.


In [20]:
## Using `in_()` for multiple values
users = session.query(User).filter(User.username.in_(['alice123', 
                                                'bob456', 'charlie789'])).all()

for user in users:
    print(f"User found: {user.name}, {user.username}")


User found: Alice, alice123
User found: Bob, bob456
User found: Charlie, charlie789


In [21]:
## not_in_()` for multiple values
users = session.query(User).filter(~User.username.in_(['admin', 'superuser'])).all()

for user in users:
    print(f"User found: {user.name}, {user.username}")


User found: Alice, alice123
User found: Bob, bob456
User found: Charlie, charlie789
User found: David, david000
User found: Eve, eve999


In [36]:
session.rollback()

In [26]:
## is_() and is_not() for exact matches

users_with_role = session.query(User).filter(User.role_id.isnot(None)).all()

for user in users_with_role:
    print(f"User found: {user.name}, {user.role.name}")

User found: Alice, Admin
User found: Bob, Admin
User found: Charlie, Editor
User found: David, Viewer
User found: Eve, Viewer


In [28]:
users_without_role = session.query(User).filter(User.role_id.is_(None)).all()
print("Users without role:")

for user in users_without_role:
    print(f"User found: {user.name}")

Users without role:


In [29]:
## Ordering results using `order_by()`
# Query all users and order by name
users = session.query(User).order_by(User.name).all()

for user in users:
    print(user.name, user.username)

Alice alice123
Bob bob456
Charlie charlie789
David david000
Eve eve999


In [30]:
# Query all users and order by name in descending order
users = session.query(User).order_by(User.name.desc()).all()

for user in users:
    print(user.name, user.username)

Eve eve999
David david000
Charlie charlie789
Bob bob456
Alice alice123


In [32]:
## counting records using `count()`
# Count the number of users
user_count = session.query(User).count()
print(f"Total users: {user_count}")

Total users: 5


In [33]:
### Example: Count users by role ID
# Count users by role_id
role_id = 1
role_user_count = session.query(User).filter(User.role_id == role_id).count()
print(f"Total users with role ID {role_id}: {role_user_count}")

Total users with role ID 1: 2


In [37]:
from sqlalchemy import func

users_by_role = (
    session.query(User.role_id, func.count(User.id))
    .group_by(User.role_id)
    .all()
)

for role_id, user_count in users_by_role:
    print(f"Total users with role ID {role_id}: {user_count}")

Total users with role ID 3: 2
Total users with role ID 2: 1
Total users with role ID 1: 2


In [ ]:
from sqlalchemy import func
from sqlalchemy.orm import joinedload

users_by_role = (
    session.query(Role.name, func.count(User.id))
    .join(User, User.role_id == Role.id)  # Join the Role table
    .group_by(Role.name)
    .all()
)

for role_, user_count in users_by_role:
    print(f"Total users with role '{role_}': {user_count}")

Total users with role 'Admin': 2
Total users with role 'Viewer': 2
Total users with role 'Editor': 1


👉 Best Practice: If you need `User.role.name` inside `.query()`, use `.join(Role)`. If you just want users with roles preloaded, use `.options(joinedload(User.role))`.

In [39]:
from sqlalchemy.orm import joinedload

users = session.query(User).options(joinedload(User.role)).all()

for user in users:
    print(f"User: {user.name}, Role: {user.role.name}")  # ✅ Works because role is preloaded

User: Alice, Role: Admin
User: Bob, Role: Admin
User: Charlie, Role: Editor
User: David, Role: Viewer
User: Eve, Role: Viewer
